# Components

> Componenents that are the building blocks to the UI

In [1]:
#| default_exp components

In [2]:
#| hide
#| export
# from fh_frankenui.foundations import *
from fasthtml.common import *
from enum import Enum, EnumType

## Utils

### Tests

In [3]:
from fastcore.test import *

In [4]:
def test_xml_eq(a, b):
    test_eq(*map(lambda x: x if isinstance(x,str) else to_xml(x), (a, b)))

### Components

In [5]:
#| export
class VEnum(Enum):
    def __add__(self, other):
        return self.__str__() + other.__str__()
    
    def __str__(self):
        base = self.__class__.__name__       
        if isinstance(self.__class__, EnumType):
            base = base.lstrip('Uk').rstrip('T')
        return f"uk-{base.lower()}-{self.value}".strip('-')

In [6]:
class UkTextT(VEnum):
    red = 'red'
    blue = 'blue'
    small = 'small'
    large = 'large'
    

test_xml_eq(Div(cls=UkTextT.red),'<div class="uk-text-red"></div>\n')
test_xml_eq(Div(cls=(UkTextT.red,UkTextT.small)),'<div class="uk-text-red uk-text-small"></div>\n')

In [7]:
#| export
def stringify(o):
    # need a better name, stringify might be too general for what it does 
    if is_listy(o): return ' '.join(map(str,o)) if o else ""
    return o.__str__()

In [8]:
assert stringify('abc') == 'abc'
assert stringify(('abc','def')) == 'abc def'
assert 'uk-input ' + stringify(()) == 'uk-input '
assert 'uk-input ' + stringify("") == 'uk-input '

## Themes

Choose a theme for your app.

In [9]:
#| export
class Theme(Enum):
    slate = "slate"
    stone = "stone"
    gray = "gray"
    neutral = "neutral"
    red = "red"
    rose = "rose"
    orange = "orange"
    green = "green"
    blue = "blue"
    yellow = "yellow"
    violet = "violet"
    zinc = "zinc"

    def headers(self):
        js = (Script(src="https://cdn.jsdelivr.net/npm/uikit@3.21.6/dist/js/uikit.min.js"),
              Script(src="https://cdn.jsdelivr.net/npm/uikit@3.21.6/dist/js/uikit-icons.min.js"))
        _url = "https://unpkg.com/franken-wc@0.0.6/dist/css/{theme}.min.css"
        return (*js, Link(rel="stylesheet", href=_url.format(theme=self.value)))


In [10]:
hdrs = Theme.blue.headers()

## Base Components

In [54]:
(Label(fr="myid")("My Label"), Select(name="myname")(Option(value="dog", content="Dog"), Option(value="cat", content="Cat")))

(['label', ('My Label',), {'for': 'myid'}],
 ['select',
  (['option', (), {'value': 'dog', 'content': 'Dog'}],
   ['option', (), {'value': 'cat', 'content': 'Cat'}]),
  {'name': 'myname'}])

In [53]:
from fasthtml.components import Uk_select, Optgroup


Uk_select(name="myname", uk_cloak=True)(
    Optgroup(label='mylabel')(*[Option(value=v, content=v.capitalize()) for v in ('dog','cat','parakeet','snake')]))

```html
<uk-select name="myname" uk-cloak>
  <optgroup label="mylabel">
    <option value="dog" content="Dog"></option>
    <option value="cat" content="Cat"></option>
    <option value="parakeet" content="Parakeet"></option>
    <option value="snake" content="Snake"></option>
  </optgroup>
</uk-select>

```

In [11]:

#| export
def UkInput(label=(), 
            lbl_cls=(),
            inp_cls=(),
            cls=('space-y-2',), # Div cls
            id="", **kwargs):
    lbl_cls, inp_cls, cls = map(stringify,(lbl_cls, inp_cls, cls))
    if label: 
        label = Label(cls='uk-form-label '+lbl_cls)(label)
        if id: label.fr = id
    res = Input(cls='uk-input '+inp_cls, **kwargs)
    if id: res.id = id
    return Div(cls=cls)(label, res)

In [12]:
inp = UkInput(label="Name", id="name"); inp

```html
<div class="">
  <label class="uk-form-label " fr="name">Name</label>
  <input class="uk-input " id="name">
</div>

```

In [13]:
#| export
class UkButtonT(VEnum):
    default = 'default'
    primary = 'primary'
    secondary = 'secondary'
    danger = 'danger'
    ghost = 'ghost'
    text = 'text'
    link = 'link'

In [14]:
#| export
def UkButton(*c, 
            cls=(), # Use UkButtonT or styles 
            **kwargs):    
    return Button(cls='uk-button ' + stringify(cls), **kwargs)(*c)

In [45]:
UkButton('App', cls=UkButtonT.primary)

TypeError: can only concatenate str (not "UkButtonT") to str

In [23]:
btn = UkButton('Click me', typ='uk-button-primary', disabled=True)
test_xml_eq(UkButton('Click me', typ='uk-button-primary', disabled=True),'<button typ="uk-button-primary" disabled class="uk-button ">Click me</button>\n')

In [24]:
test_xml_eq(UkButton(typ=(UkButtonT.primary,'w-full'))('Create Account'),'<button typ="uk-button-primary w-full" class="uk-button ">Create Account</button>\n')

In [42]:
#| export
def UkHSplit(*c, cls=(), line_cls=(), text_cls=()):
    cls, line_cls, text_cls = map(stringify,(cls, line_cls, text_cls))
    return Div(cls='relative ' + cls)(
        Div(cls="absolute inset-0 flex items-center " + line_cls)(Span(cls="w-full border-t border-border")),
        Div(cls="relative flex justify-center " + text_cls)(Span(cls="bg-background px-2 ")(*c)))



## Higher order components

In [35]:
#|export
def Card(*c, # Components that go in the body
        header=None, # Components that go in the header
        footer=None,  # Components that go in the footer
        body_cls=(), # classes for the body
        header_cls=(), # classes for the header
        footer_cls=(), # classes for the footer
        cls=(), #class for outermost component
        **kwargs # classes that for the card itself
        ):
    header_cls, footer_cls, body_cls, cls = map(stringify, (header_cls, footer_cls, body_cls, cls))
    res = []
    if header: res += [Div(cls='uk-card-header ' + header_cls)(header),]
    res += [Div(cls='uk-card-body ' + body_cls)(*c),]
    if footer: res += [Div(cls='uk-card-footer ' + footer_cls)(footer),]
    return Div(cls='uk-card '+cls, **kwargs)(*res)

In [36]:
Card(P('A simple text body'), inp, header=Strong("A bold header"), 
        footer=btn, footer_cls=('uk-text-right',))

```html
<div class="uk-card ">
  <div class="uk-card-header ">
    <strong>A bold header</strong>
  </div>
  <div class="uk-card-body ">
    <p>A simple text body</p>
    <div class="">
      <label class="uk-form-label " fr="name">Name</label>
      <input class="uk-input " id="name">
    </div>
  </div>
  <div class="uk-card-footer uk-text-right">
    <button typ="uk-button-primary" disabled class="uk-button ">Click me</button>
  </div>
</div>

```

In [37]:
Card(footer=btn, footer_cls='uk-text-right')

```html
<div class="uk-card ">
  <div class="uk-card-body "></div>
  <div class="uk-card-footer uk-text-right">
    <button typ="uk-button-primary" disabled class="uk-button ">Click me</button>
  </div>
</div>

```

In [40]:
#| hide


import nbdev; nbdev.nbdev_export()